In [3]:
import torch
from collage.model import Codon_Predictor

In [7]:
import torch

# Assume your model is something like this
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = torch.nn.Linear(10, 10)

    def forward(self, input1, input2):
        tgt_pad_mask = (input == 0).to( input1.device )
        return self.fc(input1 + input2 + tgt_pad_mask)

# Initialize the model
model = MyModel()

# Create dummy inputs for the forward method
dummy_input1 = torch.randn(1, 10)
dummy_input2 = torch.randn(1, 10)

# Wrap the inputs in a tuple
dummy_inputs = (dummy_input1, dummy_input2)

# Export the model
torch.onnx.export(model, dummy_inputs, "my_model.onnx", verbose=True)


Exported graph: graph(%onnx::Add_0 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu),
      %onnx::Add_1 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(10, 10, strides=[10, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %/Add_output_0 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu) = onnx::Add[onnx_name="/Add"](%onnx::Add_0, %onnx::Add_1), scope: __main__.MyModel:: # /tmp/ipykernel_88295/964593827.py:10:0
  %5 : Float(1, 10, strides=[10, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/fc/Gemm"](%/Add_output_0, %fc.weight, %fc.bias), scope: __main__.MyModel::/torch.nn.modules.linear.Linear::fc # /home/auberon/programming/collage/venv/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  return (%5)

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR

In [2]:
state_dict = torch.load('test.pt')

In [3]:
model = Codon_Predictor( n_input_tokens = 22, 
                         n_output_tokens = 66,
                         n_sp = 1, #len( species ),
                         model_dim = 64, 
                         ff_dim = 256,
                         n_heads = 4,
                         n_encoder_layers = 1, 
                         n_decoder_layers = 2, 
                         dropout = 0.2,
                         max_len = 500, )

In [4]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
prot = torch.tensor([[14, 13,  6,  6, 11,  1,  3, 14, 18,  1,  4, 16, 10, 11,  1,  1,  1, 15,
         15,  6,  8,  7,  2, 15, 10, 11, 10,  3, 16,  1],
        [ 3, 10,  7,  1,  1,  6,  6,  1,  3, 12, 13,  1,  6, 18,  3,  3,  9,  3,
         10, 18,  9, 20,  1,  3,  8,  6,  1, 17, 20, 20],
        [ 3,  4,  9, 17, 10, 13, 16, 10,  6,  1, 17, 10,  5, 17,  8, 10, 10, 13,
          8,  1, 10, 11, 10, 18,  9, 17,  8,  1,  4, 10],
        [10,  8, 17, 13, 12,  7, 18, 16,  5,  8,  3,  6,  8, 10, 10,  6, 10,  5,
         10, 13, 18, 15, 13, 18,  5,  1, 18, 20, 17, 16]])
cds= torch.tensor([[65, 31, 22, 56, 56, 45, 62, 55, 31, 61, 50, 59, 10,  9, 45, 62, 50, 62,
         28, 24, 56, 33, 19,  8, 28, 13, 45, 21, 55,  6],
        [65, 55, 29, 23, 50, 62, 52, 52, 58, 55, 39, 30, 58, 52, 49, 51, 51, 43,
         51, 29, 49, 43,  7, 54, 51, 33, 56, 62, 34,  7],
        [65, 51, 59, 43, 42, 25, 30, 10,  9, 56, 58, 46,  9,  5, 38, 41, 29, 25,
         22, 33, 62, 29, 45, 13, 49, 43, 46, 33, 54, 59],
        [65, 25, 33, 46, 18, 35, 23, 53,  2,  1, 33, 51, 56, 33, 13, 17, 60, 29,
          1,  9, 18, 61, 20, 26, 61,  1, 54, 49,  7, 38]])

In [6]:
model.forward(prot, cds)

PROT tensor([[14, 13,  6,  6, 11,  1,  3, 14, 18,  1,  4, 16, 10, 11,  1,  1,  1, 15,
         15,  6,  8,  7,  2, 15, 10, 11, 10,  3, 16,  1],
        [ 3, 10,  7,  1,  1,  6,  6,  1,  3, 12, 13,  1,  6, 18,  3,  3,  9,  3,
         10, 18,  9, 20,  1,  3,  8,  6,  1, 17, 20, 20],
        [ 3,  4,  9, 17, 10, 13, 16, 10,  6,  1, 17, 10,  5, 17,  8, 10, 10, 13,
          8,  1, 10, 11, 10, 18,  9, 17,  8,  1,  4, 10],
        [10,  8, 17, 13, 12,  7, 18, 16,  5,  8,  3,  6,  8, 10, 10,  6, 10,  5,
         10, 13, 18, 15, 13, 18,  5,  1, 18, 20, 17, 16]])
CDS tensor([[65, 31, 22, 56, 56, 45, 62, 55, 31, 61, 50, 59, 10,  9, 45, 62, 50, 62,
         28, 24, 56, 33, 19,  8, 28, 13, 45, 21, 55,  6],
        [65, 55, 29, 23, 50, 62, 52, 52, 58, 55, 39, 30, 58, 52, 49, 51, 51, 43,
         51, 29, 49, 43,  7, 54, 51, 33, 56, 62, 34,  7],
        [65, 51, 59, 43, 42, 25, 30, 10,  9, 56, 58, 46,  9,  5, 38, 41, 29, 25,
         22, 33, 62, 29, 45, 13, 49, 43, 46, 33, 54, 59],
        [65, 25, 

/home/auberon/programming/collage/venv/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


tensor([[[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf, -2.1312],
         ...,
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf, -2.1545,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ..., -1.0194,    -inf,    -inf]],

        [[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         ...,
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf]],

        [[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,  ...,   

In [8]:
dummy_input = (prot, cds)
input_names = ["prot", "cds"]
output_names = ["output"]
model.eval()
torch.onnx.export(model, dummy_input, "test.onnx", input_names=input_names, output_names=output_names,
                export_params=True, verbose=True)

PROT tensor([[14, 13,  6,  6, 11,  1,  3, 14, 18,  1,  4, 16, 10, 11,  1,  1,  1, 15,
         15,  6,  8,  7,  2, 15, 10, 11, 10,  3, 16,  1],
        [ 3, 10,  7,  1,  1,  6,  6,  1,  3, 12, 13,  1,  6, 18,  3,  3,  9,  3,
         10, 18,  9, 20,  1,  3,  8,  6,  1, 17, 20, 20],
        [ 3,  4,  9, 17, 10, 13, 16, 10,  6,  1, 17, 10,  5, 17,  8, 10, 10, 13,
          8,  1, 10, 11, 10, 18,  9, 17,  8,  1,  4, 10],
        [10,  8, 17, 13, 12,  7, 18, 16,  5,  8,  3,  6,  8, 10, 10,  6, 10,  5,
         10, 13, 18, 15, 13, 18,  5,  1, 18, 20, 17, 16]])
CDS tensor([[65, 31, 22, 56, 56, 45, 62, 55, 31, 61, 50, 59, 10,  9, 45, 62, 50, 62,
         28, 24, 56, 33, 19,  8, 28, 13, 45, 21, 55,  6],
        [65, 55, 29, 23, 50, 62, 52, 52, 58, 55, 39, 30, 58, 52, 49, 51, 51, 43,
         51, 29, 49, 43,  7, 54, 51, 33, 56, 62, 34,  7],
        [65, 51, 59, 43, 42, 25, 30, 10,  9, 56, 58, 46,  9,  5, 38, 41, 29, 25,
         22, 33, 62, 29, 45, 13, 49, 43, 46, 33, 54, 59],
        [65, 25, 